# Программирование на Python

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: поиск элементов на странице

### Установка и подготовка к работе

Библиотека Selenium – библиотека для управления браузером с помощью Python. Она позволяет запускать браузер и имитировать действия пользователя в нём.

Зачем это может понадобиться? Во-первых, такое «встраивание» в браузер в большинстве случаев позволяет решить проблему с возникающими капчами и иными ограничениями, так как настройки сайта не распознают, что запрос к исходному коду страницы производится автоматически. Во-вторых, необходимость имитации действий в браузере неизбежно возникает при обработке динамических веб-страниц, где некоторые элементы (окна, графики, таблицы) появляются только при определенных действиях пользователя, например, при скроллинге или наведении мышкой. В-третьих, библиотека может быть полезна в случаях, если доступ к API сайта или базы данных получить довольно сложно, но при работе в браузере информация доступна (в таком случае можем залогиниться через Python как пользователь и потихоньку выгружать данные).

Установим библиотеку:

In [ ]:
!pip install selenium

Если библиотека уже установлена, устанавливать повторно её не требуется, достаточно просто импортировать. Если библиотека установлена и вы точно знаете, что версия старая, можно запросить принудительную установку более новой версии с помощью опции `--upgrade`. 

In [ ]:
!pip install selenium --upgrade

Почему такую установку можно считать «принудительной»? Команда `pip install` действует так: проверяет наличие библиотеки на компьютере или в конкретной рабочей среде, и если хотя бы какая-то версия библиотеки присутствует, установка не производится. Поэтому, добавляя опцию `--upgrade` мы в любом случае инициализируем установку. 

Сначала импортируем библиотеку полностью, проверим, что всё идёт по плану:

In [1]:
import selenium

Проверим версию бибилиотеки, раз речь зашла о версиях:

In [2]:
selenium.__version__

'4.11.2'

Теперь импортируем из библиотеки Selenium необходимые компоненты:

* модуль `webdriver`, нужен непосредственно для запуска браузера через Python;
* коллекция атрибутов для поиска элементов на странице `By`;
* коллекция атрибутов для имитации нажатия клавиш `Keys`.

In [3]:
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

Запустим браузер Chrome средствами Selenium (в модуле есть функции для разных браузеров, но лучше всего работать с Chrome или Firefox):

In [4]:
br = wd.Chrome()

Если вы исполняете код выше первый раз, это может занять некоторое время, так как функция `Chrome()` сначала скачает и установит подходящую версию драйвера для Chrome, который будет обеспечивать связь между Selenium и той версией браузера Chrome, которая установлена у вас на компьютере. В последующие разы всё должно происходить быстрее.

В результате запуска строки выше должно открыться новое окно браузера Chrome. Оно пустое и, скорее всего, с всплывающим предупреждением о том, что браузером управляет автоматизированное тестовое ПО. Это нормально, мы «имитировали» открытие браузера от лица пользователя, теперь, если мы будем в этом окне переходить по ссылкам на сайты, сайты не будут воспринимать запросы как автоматические (в отличие от запросов через модуль `requests`, из-за автоматических или слишком быстрых запросов, собственно, могут возникать те же капчи).

Открывшееся окно браузера в течение работы закрывать нельзя. Python будет там выполнять действия, а мы будем их отслеживать со стороны. Приступим к работе.

Открываем новое окно на всю ширину экрана на случай, если какие-то элементы в маленьком окне будут мешать или накладываться друг на друга (так бывает с рекламой, всплывающими окнами и подобным):

In [5]:
br.maximize_window()

### Поиск элементов на странице

Начнём с простого примера – сайта книжного магазина «Библио-Глобус». Вообще этот сайт можно парсить и без Selenium, он пока не блокирует автоматические запросы, но зато другие сайты магазинов или онлайн-кинотеатров с похожей структурой умеют это делать.

Отправим запрос – перейдём в открытом окне по ссылке на [главную страницу](https://www.biblio-globus.ru/) сайта:

In [6]:
br.get("https://www.biblio-globus.ru/")

В окне должна открыться главная страница сайта. Найдём на странице **поле для поиска** интересующего товара. Для этого нам понадобится метод `.find_element()`, он применяется к объекту WebDriver (у нас это `br`). Вообще в Selenium есть два метода для поиска, по аналогии с методами `.find()` и `.find_all()` в `BeautifulSoup`:

* `.find_element()` – поиск одного элемента, возвращает один результат, если подходящих элементов несколько, возвращается первый;
* `.find_elements()` – поиск нескольких элементов, возвращает список результатов.

Откроем страницу в новой вкладке обычного браузера (окно, открытое через Python пока не трогаем) и зайдём в инструменты разработчика. Выберем поле для поиска и найдём соответствующий фрагмент кода HTML:

![](https://raw.githubusercontent.com/allatambov/PyAllAdd/main/selenium-one.jpeg)

Можно найти этот элемент по-разному: по тэгу `<input>`, по атрибуту `name`, по названию класса `form-control ui-autocomplete-input` или по id. Все эти методы можно найти в коллекции `By`, которую мы импортировали в начале:

* `By.TAG_NAME`: поиск по тэгу;
* `By.NAME`: поиск по атрибуту `name`;
* `By.CLASS_NAME`: поиск по названию класса.

Но из всех способов лучше выбирать самый надёжный, такой, который приведёт именно к нужному элементу, а не к элементу с таким же классом или тэгом (поле для поиска теоретически на сайте может быть не одно, да и имя `query` тоже не редкость). Поэтому остановимся на id:

In [7]:
search = br.find_element(By.ID, "SearchBooks")

Сейчас в `search` у нас хранится объект Selenium, к которому можно применять различные методы. Так как мы нашли поле для поиска, которое можно заполнять, мы воспользуемся методом `.send_keys()`, который введёт в это поле текст. Запросим книги по Python:

In [8]:
search.send_keys("Python")

В открытом окне должен появиться текст и выпадающее меню с названиями книг. Магия!

**Примечание.** Если мы нашли объект некорректно, например, не само поле (в HTML обычно с тэгом `<input>`), а рамочку вокруг него или раздел, внутри которого это поле находится, то есть те элементы, которые не подразумевают интерактива в виде ввода значений, в ответ на `.send_keys()` мы получим ошибку `element not interactable`.

Для активации поиска нужно найти соответствующую кнопку рядом с полем и кликнуть на неё. Но можно поступить проще (не всегда подобные кнопки удобно искать), ведь мы часто вместо кликания просто нажимаем на *Enter*!

In [9]:
search.send_keys(Keys.ENTER)

Отлично! Теперь нужно поработать с результатами поиска. Предположим, что мы пишем код – основу для какой-нибудь программы или приложения, которая позволяет пользователю по ключевым словам выполнить поиск и отсортировать результаты по какому-нибудь критерию, доступному на сайте. Например, по цене, от меньшей к большей. Найдём выпадающее меню с нужными опциями на странице:

![](https://raw.githubusercontent.com/allatambov/PyAllAdd/main/selenium-two.jpeg)

Так как выпадающее меню здесь одно, его можно найти просто по тэгу `<select>`:

In [10]:
select = br.find_element(By.TAG_NAME, "select")

По аналогии с предыдущим примером заполнения ввода поля, выбор опций в обычных выпадающих меню с тэгом `<select>` можно осуществить с помощью того же метода `.send_keys()`:

In [11]:
# внимание на пробелы – опция должна быть как в исходном коде страницы

select.send_keys(" По цене (от меньшей к большей) ")                                   

Отлично! Давайте ещё найдём поле для галочки «В наличии» и кликнем на него!

![](https://raw.githubusercontent.com/allatambov/PyAllAdd/main/selenium-three.jpeg)

Галочку удобнее найти по названию класса:

In [12]:
# кликаем – метод click()

tick = br.find_element(By.CLASS_NAME, "custom-control-label")
tick.click()

Технически, теперь нам достаточно извлечь исходный код страницы из открытого браузера и стандартным образом выгрузить необходимую информацию с помощью `BeautifulSoup`.

In [13]:
# .page_source – обычная строка с исходным кодом

html = br.page_source

Импортируем функцию `BeautifulSoup`:

In [14]:
from bs4 import BeautifulSoup

Преобразуем строку `html` с исходным кодом в объект `BeautifulSoup`:

In [15]:
soup = BeautifulSoup(html)

Попробуем найти карточки с товарами!

![](https://raw.githubusercontent.com/allatambov/PyAllAdd/main/selenium-four.jpeg)

Найдём все фрагменты с тэгом `<div>` и атрибутом `class` равным `product`:

In [16]:
divs = soup.find_all("div", {"class" : "product"})

# 12 товаров – у нас пока только первая страница

print(len(divs))

12


Посмотрим на пример одной карточки – одна книга, четвёртый товар в списке:

In [17]:
d = divs[3]
print(d)

<div class="product">
<a class="img_link" href="/product/10776656"><img alt="" class="img-fluid" src="https://static1.bgshop.ru/imagehandler.ashx?fileName=10776656.jpg&amp;width=200"/></a>
<div class="text">
<div class="product-sm-used-placeholder"></div>
<div class="author">Кольцов Д. М.</div>
<h3><a href="/product/10776656" id="p_title_10776656">Справочник PYTHON.  Кратко, быстро, под рукой</a></h3>
<div class="prices_item">
<div class="price_item_wrapper"><span class="price_item_title">Цена на сайте:</span> <div class="price_item_block"><s>569 ₽</s> <span class="price_item_new price_item_with_discount">512 ₽</span></div></div>
</div>
<p class="status im_status_title">
<span style="color: #3ba155;"> в наличии</span>
<span class="product-qty">3 шт.</span>
</p>
<div class="buttons row">
<div class="col-9 pl-0">
<span class="btn btn-block btn-primary" id="add_10776656" onclick="AddToBasket(10776656)">В корзину</span>
<a class="btn btn-primary add_but" href="/Basket/Detail" id="added_107

Как найти в этом фрагменте наименование товара? Найти элемент с тэгом `<h3>` и извлечь из него текст (текст внутри `<h3>...</h3>`):

In [18]:
title = d.find("h3").text
print(title)

Справочник PYTHON.  Кратко, быстро, под рукой


Во фрагменте с тэгом `<h3>` (заголовок третьего уровня, не очень крупный), помимо самого названия товара есть еще и другие элементы:

In [19]:
d.find("h3")

<h3><a href="/product/10776656" id="p_title_10776656">Справочник PYTHON.  Кратко, быстро, под рукой</a></h3>

В частности, есть ссылка на страницу товара, она заключена в тэг `<a>` внутри (ссылки обычно в такой тэг и заключаются). Выберем из найденного заголовка вложенный фрагмент с ссылкой `<a>`:

In [20]:
d.find("h3").find("a")

<a href="/product/10776656" id="p_title_10776656">Справочник PYTHON.  Кратко, быстро, под рукой</a>

Чтобы извлечь саму ссылку, без тэга и лишней информации, заберём значение из атрибута `href` с помощью метода `.get()`:

In [21]:
d.find("h3").find("a").get("href")

'/product/10776656'

Сделаем ссылку полной – доклеим к ней ссылку на главную страницу сайта:

In [23]:
link = "https://www.biblio-globus.ru" + d.find("h3").find("a").get("href")
print(link)

https://www.biblio-globus.ru/product/10776656


Чтобы найти цену товара (и обычную, и со скидкой), найдём фрагмент с тэгом `<div>` с атрибутом `class` равным `price_item_block`:

In [24]:
d.find("div", {"class" : "price_item_block"})

<div class="price_item_block"><s>569 ₽</s> <span class="price_item_new price_item_with_discount">512 ₽</span></div>

Не будем делить содержимое на части, просто выгрузим весь текст со всеми ценами и пробелами между ними:

In [25]:
d.find("div", {"class" : "price_item_block"}).text

'569\xa0₽\xa0512\xa0₽'

Неразрывные пробелы `\xa0` заменим на обычные, полную обработку цены оставим на потом (логично сначала выгрузить всё в сыром виде, а затем проанализировать, как универсальным образом преобразовать результаты; здесь, например, не у всех товаров будет две цены, так как есть товары без скидки):

In [26]:
price = d.find("div", {"class" : "price_item_block"}).text.replace("\xa0", " ")
print(price)

569 ₽ 512 ₽


Напишем функцию `get_item()`, которая будет принимать на вход одну карточку, то есть один элемент – объект `BeautifulSoup` из `divs`, а возвращать список характеристик товара:

* название товара;
* ссылка на страницу;
* цена.

In [27]:
# копируем код выше

def get_item(d):
    title = d.find("h3").text
    link = "https://www.biblio-globus.ru/" + d.find("h3").find("a").get("href")
    price = d.find("div", {"class" : "price_item_block"}).text.replace("\xa0", " ")
    return title, link, price

In [28]:
# проверяем функцию на одном элементе

get_item(divs[0])

('Папка на резинках пластиковая ErichKrause® Python Print, A4 (в пакете по 4 шт.)',
 'https://www.biblio-globus.ru//product/10730131',
 ' 159 ₽')

Применим эту функцию ко всем элементам в `divs`:

In [29]:
results = [get_item(d) for d in divs]

# несколько значений для примера
print(results[0:3])

[('Папка на резинках пластиковая ErichKrause® Python Print, A4 (в пакете по 4 шт.)', 'https://www.biblio-globus.ru//product/10730131', ' 159 ₽'), ('Папка на 4 кольцах пластиковая ErichKrause® Tropical Python, 24мм, A4 (в пакете по 4 шт.)', 'https://www.biblio-globus.ru//product/10726752', ' 269 ₽'), ('Python глазами хакера', 'https://www.biblio-globus.ru//product/10823441', '509 ₽ 458 ₽')]


Ура! Результаты из кода HTML мы выгрузили. Но давайте мыслить более глобально – результаты не помещаются на одной странице, поэтому для общего решения нам надо научиться выяснять, сколько страниц пролистывать для сбора всех товаров по ключевым словам.

Для этого нам нужно найти кнопку со стрелками `»»`, которая при кликании переносит нас на последнюю страницу результатов, и внимательно её изучить.

![](https://raw.githubusercontent.com/allatambov/PyAllAdd/main/selenium-five.jpeg)

Вариантов найти подходящий элемент, как всегда, несколько, мы найдём его по частичному тексту ссылки (частичному, поскольку внутри кнопки есть ещё элементы, посмотрите исходный код):

In [30]:
last = br.find_element(By.PARTIAL_LINK_TEXT, "»»")

Чтобы проверить, тот ли элемент мы нашли, запросим код HTML внутри `last`:

In [31]:
last.get_attribute('innerHTML')

'<span aria-hidden="true">»»</span><span class="sr-only">Next</span>'

Да, тот, но сам код нам сейчас мало интересен, нам нужна только ссылка внутри `href`, потому что именно в ней хранится информация о том, сколько страниц с результатами мы получили по нашему запросу:

In [32]:
last_link = last.get_attribute("href")
print(last_link)

https://www.biblio-globus.ru/catalog/search?query=Python&page=10&sort=5&instock=on&cat=0&isdiscount=


Вот эта информация уже гораздо полезнее, внутри этой ссылки есть указание на то, какая страница является последней. То есть, нам не придётся кликать на стрелку *Далее* до тех пор, пока мы не дойдём до конца, через какой-нибудь цикл while, мы сможем написать код, который будет пролистывать 10 страниц и забирать результаты.

Подготовим всё необходимое для цикла. Извлечём номер последней страницы:

In [33]:
# разбиваем ссылку по page=
# забираем часть после page= с индексом 1

last_link.split("page=")[1]

'10&sort=5&instock=on&cat=0&isdiscount='

In [34]:
# разбиваем результат по &
# забираем часть до первого &

last_link.split("page=")[1].split("&")[0]

'10'

In [35]:
# превращаем строку в целое число и сохраняем в n

n = int(last_link.split("page=")[1].split("&")[0])

Собственно, теперь можно через простое форматирование строк создать ссылки на страницы с результатами от 1 до 10 и запустить цикл for для извлечения всех-всех результатов.

In [ ]:
# нужна f-строка – вместо page=10 подставляем числа от 1 до 10
# https://www.biblio-globus.ru/catalog/search?query=Python&page=10&sort=5&instock=on&cat=0&isdiscount=

Импортируем функцию `sleep()` из модуля `time` для выставления задержки после каждой итерации цикл – чтобы сайт не заподозрил неладное и не зафиксировал слишком частые автоматические ззапросы:

In [36]:
from time import sleep

Создадим пустой список `res_all` и будем заполнять его в цикле, добавляя через метод `.extend()` новые списки по 12 элементов (на одной странице 12 результатов в данный момент, на последней их может быть меньше):

In [37]:
res_all = []
for i in range(1, n + 1):
    # формируем ссылку 
    # отправляем ее в адресную строку
    href = f"https://www.biblio-globus.ru/catalog/search?query=Python&page={i}&sort=5&instock=on&cat=0&isdiscount="
    br.get(href)
    
    # выгружаем код и извлекаем результаты
    html = br.page_source
    soup = BeautifulSoup(html)
    divs = soup.find_all("div", {"class" : "product"})
    results = [get_item(d) for d in divs]
    res_all.extend(results)
    # ждем 1.5 секунды перед переходом к след странице
    sleep(1.5)

In [38]:
print(res_all[10:12]) # примеры элементов списка

[('Для начинающих. Python. 12 уроков для начинающих', 'https://www.biblio-globus.ru//product/10933549', '849 ₽ 764 ₽'), ('Python. Полное руководство', 'https://www.biblio-globus.ru//product/10829190', ' 969 ₽')]


Импортируем библиотеку `pandas` и превратим список кортежей `res_all` в датафрейм (привычную таблицу):

In [39]:
import pandas as pd

In [40]:
df = pd.DataFrame(res_all)
df

,0,1,2
0,Папка на резинках пластиковая ErichKrause® Pyt...,https://www.biblio-globus.ru//product/10730131,159 ₽
1,Папка на 4 кольцах пластиковая ErichKrause® Tr...,https://www.biblio-globus.ru//product/10726752,269 ₽
2,Python глазами хакера,https://www.biblio-globus.ru//product/10823441,509 ₽ 458 ₽
3,"Справочник PYTHON. Кратко, быстро, под рукой",https://www.biblio-globus.ru//product/10776656,569 ₽ 512 ₽
4,"Python на примерах. Практика, практика и тольк...",https://www.biblio-globus.ru//product/10914448,579 ₽ 521 ₽
...,...,...,...
110,PYTHON и анализ данных: Первичная обработка да...,https://www.biblio-globus.ru//product/10911270,4719 ₽
111,Алгебра и геометрия с примерами на Python,https://www.biblio-globus.ru//product/10841699,4829 ₽
112,Предварительная подготовка данных в PYTHON. То...,https://www.biblio-globus.ru//product/10907047,6289 ₽ 4716 ₽
113,Предварительная подготовка данных в PYTHON. То...,https://www.biblio-globus.ru//product/10907046,6289 ₽ 4716 ₽


Добавим названия столбцов:

In [41]:
df.columns = ["title", "link", "price"]
df

,title,link,price
0,Папка на резинках пластиковая ErichKrause® Pyt...,https://www.biblio-globus.ru//product/10730131,159 ₽
1,Папка на 4 кольцах пластиковая ErichKrause® Tr...,https://www.biblio-globus.ru//product/10726752,269 ₽
2,Python глазами хакера,https://www.biblio-globus.ru//product/10823441,509 ₽ 458 ₽
3,"Справочник PYTHON. Кратко, быстро, под рукой",https://www.biblio-globus.ru//product/10776656,569 ₽ 512 ₽
4,"Python на примерах. Практика, практика и тольк...",https://www.biblio-globus.ru//product/10914448,579 ₽ 521 ₽
...,...,...,...
110,PYTHON и анализ данных: Первичная обработка да...,https://www.biblio-globus.ru//product/10911270,4719 ₽
111,Алгебра и геометрия с примерами на Python,https://www.biblio-globus.ru//product/10841699,4829 ₽
112,Предварительная подготовка данных в PYTHON. То...,https://www.biblio-globus.ru//product/10907047,6289 ₽ 4716 ₽
113,Предварительная подготовка данных в PYTHON. То...,https://www.biblio-globus.ru//product/10907046,6289 ₽ 4716 ₽


Выгрузим датафрейм в файл Excel – обрабатывать полученные результаты мы будем в следующей части.

In [42]:
df.to_excel("Python_items.xlsx")